In [1]:
import os
import random
import shutil
import pydicom
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from tqdm import tqdm
from scipy import ndimage
from datetime import date, datetime
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv("output_MR.csv")
icd_type = {}

for i in df.index:
    icd_type[str(df["report_uid"].loc[i])] = df["ICD"].loc[i]
    
# for i in icd_type.keys():
#     print(i, icd_type[i])
    
# del icd_type["380000002899"]

In [3]:
# path = "BRAIN MRA"
# #error_cases = []
# sorttt = []

# def create_data(icd1, icd2):
    
#     data = []
#     pbar = tqdm(total=len(os.listdir(path)))
    
#     for folder in sorted(os.listdir(path)):
        
#         if folder not in icd_type.keys():  
#             pbar.update(1)
#             continue
#         if icd_type[folder] != icd1 and icd_type[folder] != icd2:
#             pbar.update(1)
#             continue
#         if len(os.listdir(os.path.join(os.getcwd(), path, folder))) != 136:
#             #error_cases.append(folder)
#             pbar.update(1)
#             continue
   
#         slices = {}
#         for file in sorted(os.listdir(os.path.join(os.getcwd(), path, folder))):
#             ds = pydicom.dcmread(os.path.join(os.getcwd(), path, folder, file))
#             slices[ds.SliceLocation] = ds.pixel_array
        
#         keys = sorted(slices.keys(), reverse = True)
#         pixel_array = list(map(slices.get, keys))
        
#         each_patient = np.array([])  
#         for pa in pixel_array:
#             if len(each_patient) == 0:
#                 each_patient = np.array([pa.reshape(65536)])
#             else:
#                 each_patient = np.append(each_patient, [pa.reshape(65536)], axis = 0)
                
# #             print(each_patient)
            
#         each_patient_T = each_patient.T.reshape(256, 256, 136)
#         if len(data) == 0:
#             data = np.array([each_patient_T[:, :, :]])
#         else:
#             data = np.append(data, [each_patient_T[:, :, :]], axis = 0)
        
# #         for _ in range(5):
# #             data = np.append(data, [each_patient_T[:, :, 34+_:98+_]], axis = 0)
        
#         pbar.update(1)
        
#     pbar.close()  
#     return data

# nStroke_data = create_data("normal", "normal")
# stroke_data = create_data("I63", "I63")

nStroke_data = np.load("normal.npy")
stroke_data = np.load("I63.npy")

# np.save('normal', nStroke_data)
# np.save('I63', stroke_data)

print("nStroke_data.shape:", nStroke_data.shape)
print("stroke_data.shape:", stroke_data.shape)


nStroke_data.shape: (129, 256, 256, 136)
stroke_data.shape: (70, 256, 256, 136)


In [4]:
# def pltshow(data):
#     for i in range(1):
#         plt.imshow(np.squeeze(data[:, :, i]), cmap="gray")
#         plt.show()

# for _ in range(20, 30):
#     pltshow(stroke_data_r[_])

In [5]:
def normalize(data):
    _min, _max = 0, 600
    data[data < _min] = _min
    data[data > _max] = _max
    data = (data - _min) / (_max - _min)
    data = data.astype("float32")
    return data


def resize(data):
    desired_depth = 128
    desired_width = 128
    desired_height = 128
    current_depth = data.shape[-1]
    current_width = data.shape[0]
    current_height = data.shape[1]
    depth_factor = desired_depth / current_depth
    width_factor = desired_width / current_width
    height_factor = desired_height / current_height
    data = ndimage.zoom(data, (width_factor, height_factor, depth_factor), order=1)
    return data

def preprocessing(data):
    return resize(normalize(data))

stroke_data_copy = stroke_data.copy()
nStroke_data_copy = nStroke_data.copy()



stroke_data1 = np.array([preprocessing(stroke_data_copy[_,:,:,0:128]) for _ in range(stroke_data_copy.shape[0])])
nStroke_data1 = np.array([preprocessing(nStroke_data_copy[_,:,:,0:128]) for _ in range(nStroke_data_copy.shape[0])])[0:70]
print(stroke_data1.shape)
print(nStroke_data1.shape)

(70, 128, 128, 128)
(70, 128, 128, 128)


In [6]:
# f = open("error_cases.txt", "w")
# for i in error_cases:
#     f.write(i + "\n")
# f.close()

In [7]:
# 0 : stroke, 1 : not stroke
stroke_labels = np.array([0 for _ in range(len(stroke_data1))]) 
nStroke_labels = np.array([1 for _ in range(len(nStroke_data1))])

print("stroke_labels.shape[0]:", stroke_labels.shape[0])
print("nStroke_labels.shape[0]:", nStroke_labels.shape[0])

stroke_labels.shape[0]: 70
nStroke_labels.shape[0]: 70


In [8]:
X = np.concatenate((stroke_data1[:],nStroke_data1[:]), axis=0)
Y = np.concatenate((stroke_labels[:],nStroke_labels[:]), axis=0)
x_train, x_val, y_train, y_val = train_test_split(X, Y, test_size=0.3)
x_val, x_test, y_val, y_test = train_test_split(x_val, y_val, test_size=0.33)



print(x_train.shape, x_val.shape, x_test.shape)
print(y_train.shape, y_val.shape, y_test.shape)
print("", y_train, y_val, y_test, sep="\n\n")

(98, 128, 128, 128) (28, 128, 128, 128) (14, 128, 128, 128)
(98,) (28,) (14,)


[1 0 0 0 1 1 0 1 1 1 0 1 1 1 1 1 1 0 0 1 0 0 0 1 0 1 0 0 0 0 0 1 0 1 1 0 0
 0 0 0 0 1 1 0 1 0 1 0 0 0 0 1 1 0 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1
 0 1 1 0 0 0 0 0 1 0 1 1 0 1 0 0 1 0 0 0 0 0 0 0]

[1 0 0 1 0 1 1 0 1 0 1 1 1 0 0 0 1 1 1 1 1 0 0 0 1 0 0 0]

[1 1 1 0 0 0 1 1 0 1 1 1 0 0]


In [9]:
@tf.function
def rotate(volume):
    
    def scipy_rotate(volume):
        angles = [-10, -5, 0, 5, 10]
        angle = random.choice(angles)
        volume = ndimage.rotate(volume, angle, reshape=False)
        volume[volume < 0] = 0
        volume[volume > 1] = 1
        return volume

    augmented_volume = tf.numpy_function(scipy_rotate, [volume], tf.float32)
    return augmented_volume


def train_preprocessing(volume, label):
    volume = rotate(volume)
    volume = tf.expand_dims(volume, axis=-1)
    return volume, label


def validation_preprocessing(volume, label):
    volume = tf.expand_dims(volume, axis=-1)
    return volume, label

In [10]:
batch_size = 2

train_loader = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = (
    train_loader.shuffle(len(x_train))
    .map(train_preprocessing)
    .batch(batch_size)
    .prefetch(2)
)

validation_loader = tf.data.Dataset.from_tensor_slices((x_val, y_val))
validation_dataset = (
    validation_loader.shuffle(len(x_val))
    .map(validation_preprocessing)
    .batch(batch_size)
    .prefetch(2)
)

In [11]:
def get_model(width=128, height=128, depth=128):

    inputs = keras.Input((width, height, depth, 1))

    x = layers.Conv3D(filters=32, kernel_size=3, activation="relu")(inputs)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv3D(filters=64, kernel_size=3, activation="relu")(x)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)
    
    x = layers.Conv3D(filters=64, kernel_size=3, activation="relu")(x)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)
    
    x = layers.Conv3D(filters=128, kernel_size=3, activation="relu")(x)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv3D(filters=256, kernel_size=3, activation="relu")(x)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.GlobalAveragePooling3D()(x)
    x = layers.Dense(units=512, activation="relu")(x)
    x = layers.Dropout(0.3)(x)

    outputs = layers.Dense(units=1, activation="sigmoid")(x)

    # Define the model.
    model = keras.Model(inputs, outputs, name="3dcnn")
    return model


# Build model.
model = get_model(width=128, height=128, depth=128)
model.summary()

Model: "3dcnn"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128, 128, 128, 1) 0         
_________________________________________________________________
conv3d (Conv3D)              (None, 126, 126, 126, 32) 896       
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 63, 63, 63, 32)    0         
_________________________________________________________________
batch_normalization (BatchNo (None, 63, 63, 63, 32)    128       
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 61, 61, 61, 64)    55360     
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 30, 30, 30, 64)    0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 30, 30, 30, 64)    256   

In [12]:
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.0001,
    decay_steps=100000,
    decay_rate=0.96,
    staircase=True
)

optimizer = keras.optimizers.Adam(learning_rate=lr_schedule)

model.compile(
    loss="binary_crossentropy",
    optimizer=optimizer,
    metrics=["acc"],
)

# Reduce = ReduceLROnPlateau(monitor='val_loss',
#                            factor=0.1,
#                            patience=1,
#                            min_lr=0)

early_stopping = keras.callbacks.EarlyStopping(monitor="loss", patience=20, mode="min", restore_best_weights=True)

# Train the model, doing validation at the end of each epoch
epochs = 100000
model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=epochs,
    shuffle=True,
    verbose=2,
    callbacks=[early_stopping],
)


Epoch 1/100000
49/49 - 22s - loss: 0.7263 - acc: 0.4796 - val_loss: 0.9581 - val_acc: 0.5000
Epoch 2/100000
49/49 - 22s - loss: 0.6377 - acc: 0.6327 - val_loss: 1.3477 - val_acc: 0.5000
Epoch 3/100000
49/49 - 22s - loss: 0.6367 - acc: 0.6633 - val_loss: 1.9627 - val_acc: 0.5000
Epoch 4/100000
49/49 - 22s - loss: 0.6437 - acc: 0.6020 - val_loss: 2.0998 - val_acc: 0.5000
Epoch 5/100000
49/49 - 22s - loss: 0.6346 - acc: 0.7041 - val_loss: 2.4502 - val_acc: 0.5000
Epoch 6/100000
49/49 - 22s - loss: 0.6550 - acc: 0.6020 - val_loss: 2.0643 - val_acc: 0.5000
Epoch 7/100000
49/49 - 22s - loss: 0.5763 - acc: 0.7143 - val_loss: 3.2050 - val_acc: 0.5000
Epoch 8/100000
49/49 - 22s - loss: 0.6463 - acc: 0.7041 - val_loss: 1.4876 - val_acc: 0.5000
Epoch 9/100000
49/49 - 22s - loss: 0.6264 - acc: 0.6224 - val_loss: 2.6354 - val_acc: 0.5000
Epoch 10/100000
49/49 - 22s - loss: 0.5794 - acc: 0.6939 - val_loss: 2.6903 - val_acc: 0.5000
Epoch 11/100000
49/49 - 22s - loss: 0.5400 - acc: 0.7449 - val_loss: 

Epoch 87/100000
49/49 - 22s - loss: 0.0783 - acc: 0.9796 - val_loss: 1.8741 - val_acc: 0.7143
Epoch 88/100000
49/49 - 22s - loss: 0.0824 - acc: 0.9694 - val_loss: 2.0783 - val_acc: 0.6071
Epoch 89/100000
49/49 - 22s - loss: 0.0869 - acc: 0.9796 - val_loss: 2.0064 - val_acc: 0.5357
Epoch 90/100000
49/49 - 22s - loss: 0.0394 - acc: 0.9898 - val_loss: 1.8356 - val_acc: 0.6071
Epoch 91/100000
49/49 - 22s - loss: 0.0817 - acc: 0.9796 - val_loss: 1.7526 - val_acc: 0.6429
Epoch 92/100000
49/49 - 22s - loss: 0.0692 - acc: 0.9694 - val_loss: 1.3648 - val_acc: 0.6429
Epoch 93/100000
49/49 - 22s - loss: 0.0749 - acc: 0.9694 - val_loss: 1.5766 - val_acc: 0.6786
Epoch 94/100000
49/49 - 22s - loss: 0.0592 - acc: 0.9796 - val_loss: 2.2755 - val_acc: 0.5357
Epoch 95/100000
49/49 - 22s - loss: 0.0678 - acc: 0.9796 - val_loss: 1.6371 - val_acc: 0.6429
Epoch 96/100000
49/49 - 22s - loss: 0.0607 - acc: 0.9796 - val_loss: 1.7416 - val_acc: 0.6786


In [13]:
# model.save(os.path.join(
#     os.getcwd(), "H5", "TEST_" + str(date.today()) + " " + str(datetime.now().strftime("%H:%M:%S")).replace(":", "_") + ".h5"
# ))

In [14]:
cases = 14
NN = 0
NA = 0
AN = 0
AA = 0


class_names = ["stroke    ", "not stroke"]

for _ in range(cases):
    prediction = model_load.predict(np.expand_dims(x_test[_], axis=0))[0]
    scores = [1 - prediction[0], prediction[0]]
    if scores[0] < scores[1]: predict_code = 1 
    else: predict_code = 0
    if y_test[_] == 1 and predict_code == 1:
        print("Correct  ", "Actual:", class_names[y_test[_]], "Predict:", class_names[predict_code], sep="  ")
        NN += 1
    if y_test[_] == 1 and predict_code == 0:
        print("Incorrect", "Actual:", class_names[y_test[_]], "Predict:", class_names[predict_code], sep="  ")
        NA += 1
    if y_test[_] == 0 and predict_code == 1:
        print("Incorrect", "Actual:", class_names[y_test[_]], "Predict:", class_names[predict_code], sep="  ")
        AN += 1
    if y_test[_] == 0 and predict_code == 0:
        print("Correct  ", "Actual:", class_names[y_test[_]], "Predict:", class_names[predict_code], sep="  ")
        AA += 1

confusion_mat = pd.DataFrame(
    [[NN, NA], [AN, AA]], \
    index=pd.MultiIndex.from_product([["Actual"],["Normal", "Abnormal"]]), \
    columns=pd.MultiIndex.from_product([["Predict"],["Normal", "Abnormal"]])
)

confusion_mat

NameError: name 'model_load' is not defined